# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to download HTML pages from a website?
* How to extract relevant content from an HTML page? 

Furthermore, you will gain a deeper understanding of the data science lifecycle.

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) rather than [lxml](http://lxml.de/) to parse an HTML page and extract data from the page.

3. Please follow python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of this topic. I found that this is a good resource: [Tutorial: Web Scraping and BeautifulSoup](https://www.dataquest.io/blog/web-scraping-beautifulsoup/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. Your job is to extract insights from SFU data to answer questions. 

In this assignment, you will do two tasks. Please recall the high-level data science lifecycle below. I suggest that when doing this assignment, please remind yourself of what data you collected and what questions you tried to answer.


<img src="lifecycle.png" width="500">
<center><h4>Figure 1. Data Science Lifecycle</h4></center>

## Task 1: SFU CS Faculty Members

Sometimes you don't know what questions to ask. No worries. Start collecting data first. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawl Web Page

A web page is essentially a file stored in a remote machine (called web server). Please write code to download the HTML page and save it as a text file ("csfaculty.html").

In [1]:
#Task 1: Crawl web page
#a) download the HTML page and save it as a text file
#******************************************************************************

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

url = "https://www.sfu.ca/computing/people/faculty.html"
response = requests.get(url)

file = open("csfaculty.html", "w")
file.write(response.text)
file.close()

### (b) Extract Structured Data

Please write code to extract relevant content (name, rank, area, profile, homepage) from "csfaculty.html" and save them as a CSV file (like [faculty_table.csv](./faculty_table.csv)). 

In [2]:
#Task 1: Extract Structured Data
#b) Extract relevant content from "csfaculty.html" and save them as a CSV file 
#******************************************************************************

f = open("csfaculty.html", "r")
soup = bs(f, 'html.parser')      #converting to a soup object

name_rank, pro_hom = [], []
prof_names, prof_ranks, prof_interest = [], [], []
prof_profiles, prof_home = [], []

for item in soup.find_all("div", class_="text" ):   
    
    #***************Name, rank*********************
    for h4 in item.find_all('h4'):
        if h4.text == '\xa0':
            pass
        else:
            name_rank = h4.text.split(',')
            if 'director' in str(name_rank[0]).lower():
                pass
            else:
                prof_names.append(name_rank[0])                
                clean_ranks = str(name_rank[1]).replace('\n', '').replace('\xa0','')
                prof_ranks.append(clean_ranks)
                
    #*******************Area**************************
    for p in item.find_all('p'):
        line = p.text 
        if 'Area' in line:
            prof_interest.append(line.split("Area:")[1].replace('\n', '').replace('\xa0',''))
    
    #*******************Profile***********************   
    for link in item('a'):
        str_link = str(link.get('href'))
        if "people/faculty" in str_link and 'author' not in str_link:
            if str_link.startswith('http:'):
                prof_profiles.append(str_link)
            else:
                link1 = "http://www.sfu.ca" + str_link   #getting the link in openable format
                prof_profiles.append(link1)
        prof_profiles = list(dict.fromkeys(prof_profiles))
        
                
    #*******************Homepage********************** 
    a4=[]
    for t in item('a'):
        a4.append(t.get('href'))  
    if (len(a4)  <= 3):
        pro_hom.append(a4)

for x in pro_hom:
    if len(x) == 1:
        prof_home.append(' ')
    elif len(x) == 2:
        prof_home.append(x[1])
    elif len(x) == 3:
        prof_home.append(x[2])
        
    #*******************special cases****************
prof_interest.insert(20, 'COMPUTATIONAL LINGUISTICS AND LOGIC PROGRAMMING')
prof_ranks[20] = 'Professor'
prof_names.pop(64), prof_ranks.pop(64), prof_home.pop(64)

    #*******************to dataframe and csv*********
df = pd.DataFrame({"Name": prof_names, "Rank": prof_ranks, "Area": prof_interest, "Profile": prof_profiles, "Homepage": prof_home})
df.to_csv("faculty_table.csv", index = False)
print("end")

end


### (c) Interesting Finding

Note that you don't need to do anything for Task 1(c). The purpose of this part is to give you some sense about how to leverage exploratory data analysis (EDA) to come up with interesting questions about the data. EDA is an important topic in data science; you will  learn it soon from this course. 


First, please install [dataprep](http://dataprep.ai).
Then, run the cell below. 
It shows a bar chart for every column. What intersting findings can you get from these visualizations? 

In [3]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_table.csv")
plot(df)

ModuleNotFoundError: No module named 'dataprep'

Below are some examples:

**Finding 1:** Professor# (29) is more than 3x larger than Associate Professor# (9). 

**Questions:** Why did it happen? Is it common in all CS schools in Canada? Will the gap go larger or smaller in five years? What actions can be taken to enlarge/shrink the gap?


**Finding 2:** Homepage has 20.3% missing values. 

**Questions:** Why are there so many missing values? Is it because many faculty do not have their own homepages or do not add their homepages to the school page? What actions can be taken to avoid this to happen in the future? 

## Task 2: Age Follows Normal Distribution?

In this task, you start with a question and then figure out what data to collect.

The question that you are intersted in is `Does SFU CS faculty age follow a normal distribution?`

To estimate the age of a faculty member, you can collect the year in which s/he graduates from a university (`gradyear`) and then estimate `age` using the following equation:

$$age \approx 2020+23 - gradyear$$

For example, if one graduates from a university in 1990, then the age is estimated as 2020+23-1990 = 53. 



### (a) Crawl Web Page

You notice that faculty profile pages contain graduation information. For example, you can see that Dr. Jiannan Wang graduated from Harbin Institute of Technology in 2008 at [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please write code to download the 64 profile pages and save each page as a text file. 

In [4]:
#Task 2: Crawling Web Page
#a) Code to download the 64 profile pages and save each page as a text file
#******************************************************************************

for i in range(0,len(prof_profiles)):
    pages = requests.get(prof_profiles[i]) #fetching contents from the list of profiles (list prof_profiles from previous cell)
    to_text = pages.text
    file = open(str(i+1) + "_profile.txt","w", encoding='utf-8') #naming the files as '#_profile.txt'
    file.write(to_text)
    file.close()

print("end")

end


### (b) Extract Structured Data

Please write code to extract the earliest graduation year (e.g., 2008 for Dr. Jiannan Wang) from each profile page, and create a csv file like [faculty_grad_year.csv](./faculty_grad_year.csv). 

In [5]:
#Task 2: Extract Structured Data
#b) Code to extract the earliest graduation year from each profile page, and create a csv
#******************************************************************************

from urllib.request import urlopen
import re

result = []
nf = "Not found"
for i in range(0,len(prof_profiles)):
    html = urlopen(prof_profiles[i])
    soup = bs(html, 'html.parser') #converting object (url link) to type "soup"
    
    for item in soup.find_all("div", class_="text parbase section" ):   
        if(item.h2):
            if("Education" in (item.h2.text)):
                if(item.find('ul')):
                    tag_contents = str(item.h2.find_next("ul").text)
                    nf = tag_contents
                else:
                    tag_contents = str(item.h2.find_next("p").text)
                    nf = tag_contents
            else: 
                for x in item.find_all('p'):
                    nf += x.text

    list_years = re.findall('[1-9][0-9][0-9][0-9]', nf)   #extracting all the years from the string 
    list_years = [ int(x) for x in list_years ]  #convert each element of list to integer
    list_years.sort()
    
    if len(list_years) > 0:
        result.append(list_years[0])
    else:
        result.append('')

df = pd.DataFrame({"Name": prof_names, "Gradyear": result})
df.to_csv("faculty_grad_year.csv", index = False)
print("end")

end


### (c) Interesting Finding

Similar to Task 1(c), you don't need to do anything here. Just look at different visualizations w.r.t. age and give yourself an answer to the question: `Does SFU CS faculty age follow a normal distribution?`

In [10]:
from dataprep.eda import plot
import pandas as pd
from matplotlib import pyplot

df = pd.read_csv("faculty_grad_year.csv")
df["age"] = 2020+23-df["gradyear"]

plot(df, "age")
pyplot.hist(df["age"])
pyplot.show()

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:829: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:830: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


<Figure size 640x480 with 1 Axes>

## Submission

Complete the code in this [notebook](https://github.com/sfu-db/bigdata-cmpt733/blob/master/Assignments/A1/A1.ipynb), and submit it to the CourSys activity `Assignment 1`.

<hr/>
## <span style="color:red">Bonus: Contribute to dataprep</span>

**Total Bonus: 0.2 + 0.3 = 0.5**

1. If you create an issue (i.e., report a bug or request a feature) at [link1](https://github.com/sfu-db/dataprep/issues) or [link2](https://github.com/sfu-db/DataConnectorConfigs/issues), you will get **0.2** bonus points.

2.  Creating more issues will *not* give you more bonus points, but you are encouraged to do so for learning more.

2. If you send a pull request (i.e., fix a bug or implement a feature or add data connector for a new website) at [link1](https://github.com/sfu-db/dataprep/pulls) or [link2](https://github.com/sfu-db/DataConnectorConfigs/pulls) and the pull request gets merged into the repo, you will get **0.3** bonus points. 

4.  Sending more pull requests will *not* give you more bonus points, but you are encouraged to do so for learning more.

5. These bonus points will be directly added to your final grade.






**How to Submit**
* Submit github link(s) to the CourSys activity `Bonus`.
* Due on `March 15, 2020`

If you love dataprep, please support it by simply clicking the **<span style="color:red">Star</span>** on [Github](https://github.com/sfu-db/dataprep).
<img src="dataprep-star.png" width="1000">